In [1]:
import pandas as pd
import numpy as np

In [5]:
df_cluster_0 = pd.read_csv("../output/merged_clustered/cluster_0_combined.csv")

In [6]:
df_cluster_0.head()

,prcp(mm/day),tmax(C),tmin(C),tavg(C),srad_lw(w/m2),srad_sw(w/m2),wind_u(m/s),wind_v(m/s),wind(m/s),rel_hum(%),...,umap_40d_dim_34,umap_40d_dim_35,umap_40d_dim_36,umap_40d_dim_37,umap_40d_dim_38,umap_40d_dim_39,umap_40d_dim_40,cluster,gauge_id,baseflow
0,0.0,29.59,14.28,21.94,338.99,297.96,1.32,-0.49,1.41,55.59,...,5.1424,6.673698,2.740438,8.138597,6.841072,4.864698,8.100951,0,4005,NaN
1,0.0,29.45,14.40,21.92,346.42,295.43,1.83,-0.12,1.83,54.63,...,5.1424,6.673698,2.740438,8.138597,6.841072,4.864698,8.100951,0,4005,NaN
2,0.0,29.62,14.87,22.24,345.24,291.91,2.18,-0.31,2.20,57.00,...,5.1424,6.673698,2.740438,8.138597,6.841072,4.864698,8.100951,0,4005,NaN
3,0.0,29.48,15.25,22.36,355.33,294.54,0.00,0.16,0.16,51.74,...,5.1424,6.673698,2.740438,8.138597,6.841072,4.864698,8.100951,0,4005,NaN
4,0.0,29.59,15.68,22.64,358.81,293.83,-0.28,0.89,0.93,54.56,...,5.1424,6.673698,2.740438,8.138597,6.841072,4.864698,8.100951,0,4005,NaN


In [7]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [8]:
s = len(merged_df)

In [9]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

X = merged_df.drop('3005', axis=1) 
y = merged_df['3005']

X_train = X[:round(s*0.8)]
y_train = y[:round(s*0.8)]

X_test = X[round(s*0.9):]
y_test = y[round(s*0.9):]

X_val = X[round(s*0.8):round(s*0.9)]
y_val = y[round(s*0.8):round(s*0.9)]


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)


In [10]:
X_test.shape, y_test.shape

((1389, 220), (1389,))

In [11]:
# X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
# X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
# y_train_tensor = torch.from_numpy(y_train.to_numpy().astype(np.float32))
# y_test_tensor = torch.from_numpy(y_test.to_numpy().astype(np.float32))

In [12]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]
  
class SequenceDataset(Dataset):
    def __init__(self, features, labels, sequence_length = 20):
        """
        features: shape (N, num_features)
        labels: shape (N,) or (N, 1)
        sequence_length: number of time steps in each input sequence
        """
        self.sequence_length = sequence_length

        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels.to_numpy(), dtype=torch.float32)

        # Ensure labels have shape (N, 1)
        if self.labels.ndim == 1:
            self.labels = self.labels.unsqueeze(1)

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        x = self.features[idx : idx + self.sequence_length]  # shape: (seq_len, num_features)
        y = self.labels[idx + self.sequence_length]          # target: next step
        return x, y


In [13]:
train_dataset = SequenceDataset(X_train, y_train)
test_dataset = SequenceDataset(X_test, y_test)
val_dataset = SequenceDataset(X_val, y_val)

train_loader = DataLoader(
                    train_dataset,
                    batch_size=32,
                    shuffle=True)

test_loader = DataLoader(
                    test_dataset,
                    batch_size=32,
                    shuffle=True)
val_loader = DataLoader(
                    val_dataset,
                    batch_size=32,
                    shuffle=True
)




In [14]:
# # Map static features to initial hidden and cell state
# self.static_to_h = nn.Linear(static_size, hidden_size * num_layers)
# self.static_to_c = nn.Linear(static_size, hidden_size * num_layers)

# # Map static -> (batch_size, hidden_size * num_layers)
# h0 = self.static_to_h(x_static).view(self.num_layers, batch_size, self.hidden_size)
# c0 = self.static_to_c(x_static).view(self.num_layers, batch_size, self.hidden_size)
        
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, num_layers):
        super().__init__()

        self.lstm1 = nn.LSTM(input_size, 256, num_layers, batch_first=True)
        self.dropout1 = nn.Dropout(0.5)
        self.relu1 = nn.ReLU()

        self.lstm2 = nn.LSTM(256, 128, num_layers, batch_first=True)
        self.dropout2 = nn.Dropout(0.5)
        self.relu2 = nn.ReLU()

        self.lstm3 = nn.LSTM(128, 32, num_layers, batch_first=True)
        self.dropout3 = nn.Dropout(0.5)
        self.relu3 = nn.ReLU()

        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout1(out)
        out = self.relu1(out)

        out, _ = self.lstm2(out)
        out = self.dropout2(out)
        out = self.relu2(out)

        out, _ = self.lstm3(out)
        out = self.dropout3(out)
        out = self.relu3(out)

        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        return out


In [15]:
num_features = X_train.shape[1]
model = LSTMModel(input_size=num_features, num_layers=1)
model.to("cuda")


LSTMModel(
  (lstm1): LSTM(220, 256, batch_first=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (relu1): ReLU()
  (lstm2): LSTM(256, 128, batch_first=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (relu2): ReLU()
  (lstm3): LSTM(128, 32, batch_first=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (relu3): ReLU()
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [16]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: akshatshaw47 (akshatshaw47-iit-roorkee). Use `wandb login --relogin` to force relogin


True

In [17]:

wandb.init(project="spark'25-lstm-testing", 
    name = 'test3',
    config={
    "epochs": 1000,
    "batch_size": 64,
    "lr": 0.0001,
    "optimizer": "Adam",
    "loss_fn": "MSELoss",
    "model": "LSTM"
})


In [18]:
class LogRMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps  # to avoid log(0)

    def forward(self, pred, target):
        # Clamp values to avoid log(0)
        pred = torch.clamp(pred, min=self.eps)
        target = torch.clamp(target, min=self.eps)

        # Compute log RMSE
        log_pred = torch.log(pred)
        log_target = torch.log(target)
        loss = torch.sqrt(torch.mean((log_pred - log_target) ** 2))
        return loss


In [19]:
# criterion = nn.MSELoss()
criterion = LogRMSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Training loop with validation
for epoch in range(1000):
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    wandb.log({"epoch": epoch+1, "train_loss": train_loss, "val_loss": val_loss})

Epoch 1, Train Loss: 6.8546, Val Loss: 3.2792
Epoch 2, Train Loss: 3.0306, Val Loss: 2.6012
Epoch 3, Train Loss: 2.6766, Val Loss: 2.4457
Epoch 4, Train Loss: 2.5244, Val Loss: 2.3500
Epoch 5, Train Loss: 2.4293, Val Loss: 2.2792
Epoch 6, Train Loss: 2.3718, Val Loss: 2.2215
Epoch 7, Train Loss: 2.3143, Val Loss: 2.1644
Epoch 8, Train Loss: 2.2571, Val Loss: 2.1142
Epoch 9, Train Loss: 2.1916, Val Loss: 2.0604
Epoch 10, Train Loss: 2.1497, Val Loss: 2.0120
Epoch 11, Train Loss: 2.1116, Val Loss: 1.9696
Epoch 12, Train Loss: 2.0508, Val Loss: 1.9264
Epoch 13, Train Loss: 2.0179, Val Loss: 1.8824
Epoch 14, Train Loss: 1.9636, Val Loss: 1.8406
Epoch 15, Train Loss: 1.9198, Val Loss: 1.8022
Epoch 16, Train Loss: 1.8827, Val Loss: 1.7635
Epoch 17, Train Loss: 1.8512, Val Loss: 1.7281
Epoch 18, Train Loss: 1.8155, Val Loss: 1.6889
Epoch 19, Train Loss: 1.7731, Val Loss: 1.6578
Epoch 20, Train Loss: 1.7421, Val Loss: 1.6277
Epoch 21, Train Loss: 1.7071, Val Loss: 1.6002
Epoch 22, Train Loss: 

KeyboardInterrupt: 